# CS471 Assignment 3: Naive Bayes

1.   Aiman Madan
https://colab.research.google.com/drive/1Z-iHvDKqA-_fjymZIo2adV4QsHq0egF1?usp=sharing

# Load the dataset and split into training and testing sets (first 20 into training and the rest into testing)

In [ ]:
# Load necessary libraries
import pandas as pd
from collections import defaultdict


!wget 'https://drive.google.com/uc?export=download&id=1MJzF39zlYnTkH4SfvW0TcTVXTrTAeDP-' -O dataset.csv


# Load the dataset, skip the header row
df = pd.read_csv('dataset.csv', header=None, skiprows=1, names=["Target", "data"])

# Training
print("Training set:")
train_df = df[:20]
print(train_df)
# Testing
print("\nTesting set:")
test_df = df[20:]
print(test_df)


--2024-10-04 03:48:58--  https://drive.google.com/uc?export=download&id=1MJzF39zlYnTkH4SfvW0TcTVXTrTAeDP-
Resolving drive.google.com (drive.google.com)... 142.251.8.139, 142.251.8.101, 142.251.8.102, ...
Connecting to drive.google.com (drive.google.com)|142.251.8.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1MJzF39zlYnTkH4SfvW0TcTVXTrTAeDP-&export=download [following]
--2024-10-04 03:48:58--  https://drive.usercontent.google.com/download?id=1MJzF39zlYnTkH4SfvW0TcTVXTrTAeDP-&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.97.132, 2404:6800:4008:c00::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.97.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008 (2.0K) [application/octet-stream]
Saving to: ‘dataset.csv’

dataset.csv         100%[===================>]   1.96K  --.-KB/s    in 0s 

# Compute the prior probabilities: P(spam) and P(ham)

In [ ]:
# Calculate the number of spam and ham emails in the training set
spam_messages = len(train_df[train_df['Target'] == 'spam'])
print("Number of spam messages:", spam_messages)
ham_messages = len(train_df[train_df['Target'] == 'ham'])
print("Number of ham messages:", ham_messages)

# Calculate the prior probabilities
p_spam = spam_messages / len(train_df)
print("P(spam):", p_spam)
p_ham = ham_messages / len(train_df)
print("P(ham):", p_ham)


Number of spam messages: 9
Number of ham messages: 11
P(spam): 0.45
P(ham): 0.55


# Compute the conditional probabilities P(sentence/spam)

In [ ]:

# Initialize word dictionary with counts for spam and ham
word_dict = defaultdict(lambda: {'spam': 0, 'ham': 0})

# Tokenize and count words in spam and ham messages
for _, row in train_df.iterrows():
    words = row['data'].split()
    label = row['Target']
    for word in words:
        word_dict[word][label] += 1

# Apply Laplace smoothing and calculate total words for spam and ham
unique_words = len(word_dict)
total_spam_words = sum(word_dict[word]['spam'] for word in word_dict) + unique_words  # Laplace smoothing
total_ham_words = sum(word_dict[word]['ham'] for word in word_dict) + unique_words    # Laplace smoothing



# Calculate conditional probabilities with Laplace smoothing
for word in word_dict:
    word_dict[word]['spam'] = (word_dict[word]['spam'] + 1) / total_spam_words
    word_dict[word]['ham'] = (word_dict[word]['ham'] + 1) / total_ham_words

    print(f"P({word}|spam): {word_dict[word]['spam']:.6f}, P({word}|ham): {word_dict[word]['ham']:.6f}")




P(WINNER|spam): 0.005882, P(WINNER|ham): 0.003891
P(As|spam): 0.005882, P(As|ham): 0.003891
P(a|spam): 0.014706, P(a|ham): 0.011673
P(valued|spam): 0.005882, P(valued|ham): 0.003891
P(network|spam): 0.005882, P(network|ham): 0.003891
P(customer|spam): 0.005882, P(customer|ham): 0.003891
P(you|spam): 0.014706, P(you|ham): 0.023346
P(have|spam): 0.008824, P(have|ham): 0.007782
P(been|spam): 0.005882, P(been|ham): 0.007782
P(selected|spam): 0.005882, P(selected|ham): 0.003891
P(to|spam): 0.026471, P(to|ham): 0.007782
P(receive|spam): 0.005882, P(receive|ham): 0.003891
P(prize|spam): 0.005882, P(prize|ham): 0.003891
P(reward|spam): 0.005882, P(reward|ham): 0.003891
P(To|spam): 0.008824, P(To|ham): 0.003891
P(claim|spam): 0.005882, P(claim|ham): 0.003891
P(call|spam): 0.017647, P(call|ham): 0.007782
P(If|spam): 0.005882, P(If|ham): 0.003891
P(had|spam): 0.005882, P(had|ham): 0.003891
P(your|spam): 0.014706, P(your|ham): 0.007782
P(mobile|spam): 0.011765, P(mobile|ham): 0.003891
P(for|spam):

# Compute the posterior probabilities:

(probability of a sentence belonging to a spam or ham)

P(spam/sentence) ∝ P(spam) * P(sentence/spam)

Posterior ∝ prior * conditional

P(ham/sentence) ∝ P(ham) * P(sentence/ham)


**Display the sentence**

Print the posterior probability of a sentence belonging to spam or ham

Display the class (spam or ham)

In [ ]:
# Classify each sentence in the test set and compute accuracy
correct_predictions = 0
for index, row in test_df.iterrows():
    words = row['data'].split()

    # Calculate posterior probabilities
    spam_prob = p_spam
    ham_prob = p_ham

    for word in words:
        spam_prob *= word_dict[word]['spam'] if word in word_dict else 1 / total_spam_words
        ham_prob *= word_dict[word]['ham'] if word in word_dict else 1 / total_ham_words

    # Predicted class
    predicted_class = "spam" if spam_prob > ham_prob else "ham"

    # Display sentence, probabilities, and predicted class
    print(f"Sentence: {row['data']}")
    print(f"P(spam|sentence): {spam_prob:.6e}, P(ham|sentence): {ham_prob:.6e}")
    print(f"Predicted Class: {predicted_class}\n")

    # Track correct predictions for accuracy
    if predicted_class == row["Target"]:
        correct_predictions += 1

Sentence: Tell where you reached
P(spam|sentence): 1.683708e-10, P(ham|sentence): 7.564521e-10
Predicted Class: ham

Sentence: Your gonna have to pick up a burger for yourself on your way home
P(spam|sentence): 7.707428e-33, P(ham|sentence): 3.851503e-32
Predicted Class: ham

Sentence: As a valued customer I am pleased to advise you that for your recent review you are awarded a Bonus Prize
P(spam|sentence): 1.956572e-47, P(ham|sentence): 1.685169e-46
Predicted Class: ham

Sentence: Urgent you are awarded a complimentary trip to EuroDisinc To claim text immediately
P(spam|sentence): 2.994886e-30, P(ham|sentence): 1.855943e-30
Predicted Class: spam

Sentence: Finished class where are you
P(spam|sentence): 9.904166e-13, P(ham|sentence): 5.886787e-12
Predicted Class: ham

Sentence: where are you how did you perform 
P(spam|sentence): 4.283809e-17, P(ham|sentence): 4.812889e-15
Predicted Class: ham

Sentence: you can call me now
P(spam|sentence): 8.913750e-12, P(ham|sentence): 2.354715e-11


# Report the test set accuracy

Accuracy = no. of sentences correctly predicted by model / total sentences

In [ ]:
# Calculate and print accuracy
accuracy = correct_predictions / len(test_df)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 90.00%
